In [18]:
from deep_translator import MicrosoftTranslator
from tqdm import tqdm
import translators as ts
import numpy as np
import pandas as pd
import os

In [9]:
drop_columns = ['answer',
 'ccid',
 'context_id',
 'course_id',
 'course_order',
 'create_time',
 'end',
 'enroll_time',
 'exercise_id',
 'gender',
 'graph_predict',
 'ground_truth',
 'id',
 'language',
 'location',
 'name_en',
 'problem_id',
 'resource_id',
 'score',
 'sign',
 'start',
 'text_predict',
 'type',
 'user_id',
 'year_of_birth']
drop_keys = ['resource_id', 'chapter']

In [10]:
def save_file(df, name):
    if os.path.isdir('translated') is False:
        os.mkdir('translated')
    df.to_json(f'translated/{name}_translated.json', orient='records', force_ascii=False)
    return df

In [11]:
def translate_text_new(texts, type_trans):
    if len(texts) != 0:
        if type_trans == 'dt-microsoft':
            texts = MicrosoftTranslator(api_key='823b2fcb911f41b0960aafaa0cf3c1aa', region='southeastasia',
                                        source='zh-hans', target='vi').translate_batch(list(texts))
        elif type_trans == 'dt-mymemory':
            pass
    return texts


In [12]:
def translate_list_new(lst, type_trans):
    if len(lst) != 0:
        valid_data = [item for item in lst if item is not None]
        random_sample = valid_data[0]

        type_rnd = type(random_sample).__name__
        if type_rnd == 'list':
            lst = [translate_text_new(item, type_trans) for item in lst]
        elif type_rnd == 'dict':
            lst = [translate_dict_new(item, type_trans) for item in lst]
        else:
            pass
    return lst


In [13]:
def translate_dict_new(dct, type_trans):
    lst_keys = list(dct.keys())

    if len(lst_keys) != 0:
        lst_keys = np.setdiff1d(lst_keys, drop_keys)
        for item in lst_keys:
            type_keys_dict = type(dct[item]).__name__

            if type_keys_dict == 'list':
                dct[item] = translate_list_new(dct[item], type_trans)
            elif type_keys_dict == 'str':
                dct[item] = translate_text_new(dct[item], type_trans)
    return dct

In [14]:
def translate_data(df, name, type_trans, checkpoint=None, startpos=0, block=10):
    if os.path.exists(f'{name}_translated.json'):
        ans = input(f"Do you want to overwrite the exist file: {name}_translated.json? (Y/N)")
        if ans.lower() != 'y': return df

    if checkpoint is None: checkpoint = len(df)
    old_idx = startpos

    col_df = np.setdiff1d(np.array(df.columns), drop_columns)
    col_types = df[col_df].apply(lambda col: type(col[0])).to_numpy()
    col_df_dict = dict(zip(col_df, [col_type.__name__ for col_type in col_types]))

    print("Start pos:", startpos)
    for idx in tqdm(range(startpos, len(df), block), desc="Translating..."):
        for col_name, col_type in col_df_dict.items():
            if col_type == 'str':
                df[col_name].iloc[idx:idx + block] = translate_text_new(df[col_name].iloc[idx:idx + block], type_trans)
            elif col_type == 'list':
                print("Hi")
                df[col_name].iloc[idx:idx + block] = translate_list_new(df[col_name].iloc[idx:idx + block], type_trans)
            elif col_type == 'dict':
                df[col_name].iloc[idx:idx + block] = translate_dict_new(df[col_name].iloc[idx:idx + block], type_trans)
            else:
                continue

        if (idx - startpos + 1) % checkpoint == 0:
            df.iloc[old_idx:idx + block] = save_file(df, name).iloc[old_idx:idx + block]
            old_idx = idx

    df.iloc[old_idx:] = save_file(df, name).iloc[old_idx:]
    return df

In [15]:
name = 'example'
folder = ''
folder = folder + '/' if folder != '' else '' 
df = pd.read_json(f'{folder}{name}.json', lines=True)
df

,id,name,name_en,sign,about,motto
0,S_1,清华大学,Tsinghua University,thu,简称“清华”，由中华人民共和国教育部直属，中央直管副部级建制，位列“211工程”、“985工...,"自强不息,厚德载物"
1,S_2,北京大学,Peking University,PKU,北京大学（Peking University），简称“北大”，是中华人民共和国教育部直属的全...,博学、审问、慎思、明辨
2,S_3,武汉大学,Wuhan University,whu,武汉大学（Wuhan University）简称“武大”，是中华人民共和国教育部直属的综合性...,自强 弘毅 求是 拓新
3,S_4,苏州大学,Soochow University,suda,苏州大学（Soochow University），坐落于历史文化名城苏州，是教育部与江苏省人...,养天地正气，法古今完人
4,S_5,四川大学,Sichuan University,scu,四川大学（Sichuan University），简称“川大”，坐落于四川省会成都，是教育部...,
...,...,...,...,...,...,...
424,S_1402,高雄医学大学,Kaohsiung Medical University,Kaohsiung Medical University,高雄医学大学（Kaohsiung Medical University）源自1954年创校的...,
425,S_1405,黑龙江农垦职业学院,Heilongjiang Agricultural ReclamationVocationa...,nkzy,黑龙江农垦职业学院（Heilongjiang Agricultural Reclamatio...,
426,S_1448,江西财经大学,Jiangxi University of Finance And Economics,jxufe,江西财经大学（Jiangxi University of Finance and Econo...,
427,S_1453,贵州交通职业技术学院,GuiZhou Communications Polytechnic,gzjtzy,贵州交通职业技术学院是一所以交通为特色的理工类高职院校。创办于1958年，走过国家“示范校”...,知行合一 德技双馨


In [19]:
translate_data(df, name, 'dt-microsoft', checkpoint=1, startpos=0, block=10)

Start pos: 0


Translating...: 100%|██████████████████████████████████████████████████████████████████| 43/43 [10:19<00:00, 14.40s/it]


,id,name,name_en,sign,about,motto
0,S_1,Đại học Thanh Hoa,Tsinghua University,thu,"Được gọi là ""Thanh Hoa"", trực thuộc Bộ Giáo dụ...","Hoàn thiện bản thân, đức hạnh"
1,S_2,Đại học Bắc Kinh,Peking University,PKU,"Đại học Bắc Kinh, được gọi là ""Đại học Bắc Kin...","Uyên bác, tò mò, chu đáo, sáng suốt"
2,S_3,Đại học Vũ Hán,Wuhan University,whu,"Đại học Vũ Hán, được gọi là ""Đại học Vũ Hán"", ...","Tự hoàn thiện, Hongyi, tìm kiếm sự thật, đổi mới"
3,S_4,Đại học Soochow,Soochow University,suda,"Đại học Soochow, nằm ở thành phố lịch sử và vă...","Hãy vun trồng sự công bình của trời đất, và lu..."
4,S_5,Đại học Tứ Xuyên,Sichuan University,scu,"Đại học Tứ Xuyên, được gọi là ""Đại học Tứ Xuyê...",
...,...,...,...,...,...,...
424,S_1402,Đại học Y Cao Hùng,Kaohsiung Medical University,Kaohsiung Medical University,Đại học Y khoa Cao Hùng (Đại học Y Cao Hùng) c...,
425,S_1405,Cao đẳng nghề khai hoang nông nghiệp Hắc Long ...,Heilongjiang Agricultural ReclamationVocationa...,nkzy,Trường Cao đẳng nghề Khai hoang Nông nghiệp Hắ...,
426,S_1448,Đại học Tài chính và Kinh tế Giang Tây,Jiangxi University of Finance And Economics,jxufe,Đại học Tài chính và Kinh tế Giang Tây (Đại họ...,
427,S_1453,Cao đẳng kỹ thuật và dạy nghề vận tải Quý Châu,GuiZhou Communications Polytechnic,gzjtzy,Cao đẳng nghề và kỹ thuật Giao thông Quý Châu ...,"Sự thống nhất của kiến thức và hành động, cả đ..."
